<a href="https://colab.research.google.com/github/pushpraj-vulcan/pilot-project/blob/main/MP_BigData_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import geemap.geemap as map
import ee
ee.Authenticate()
ee.Initialize(project='ee-pushprajsingh')
m = map.Map()

In [12]:
MP = ee.FeatureCollection('projects/ee-pushprajsingh/assets/MP_STATE_BOUNDARY')
m.addLayer(MP, {}, 'Madhya Pradesh')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [13]:
vis_para = {
    'min': 0,
    'max': 100,
    'palette': ['#006400', '#ffbb22', '#ffff4c', '#f096ff', '#fa0000', '#b4b4b4', '#f0f0f0', '#0064c8', '#0096a0', '#00cf75', '#fae6a0']}


In [14]:
legend = {'Treecover': '#006400',
          'Shrubland': '#ffbb22',
          'Grassland': '#ffff4c',
          'Croplands': '#f096ff',
          'Built-up': '#fa0000',
          'Bare/Sparse vegetation': '#b4b4b4',
          'Snow-ice': '#f0f0f0',
          'Permanent water bodies': '#0064c8',
          'Herbaceous wetland': '#0096a0',
          'Mangroves': '#00cf75',
          'Moss and Lichen': '#fae6a0'
          }

In [15]:
MP_esa = ee.ImageCollection('ESA/WorldCover/v200').mean().clip(MP).select('Map')
m.addLayer(MP_esa, vis_para, 'ESA WorldCover');
m.add_legend(Title = "Legend", legend_dict=legend,**vis_para)

In [16]:
landsat8 = ee.ImageCollection("LANDSAT/LC08/C02/T1").filterBounds(MP).filterDate('2022-01-01','2023-01-01').filterMetadata('CLOUD_COVER', 'less_than', 5).mean().clip(MP)


In [17]:
TrainingP = MP_esa.sample(**{
    'region': MP,
    'scale': 60,
    'numPixels': 4000,
    'seed': 1,
    'geometries': True
})
m.addLayer(TrainingP, {}, 'Training Points')

In [18]:
bands = ['B2', 'B3', 'B4', 'B5', 'B6']
label = 'Map'
training = landsat8.select(bands).sampleRegions(**{
    'collection': TrainingP,
    'properties': [label],
    'scale': 60,
})
trained = ee.Classifier.smileRandomForest(10).train(training, label, bands)
result = landsat8.select(bands).classify(trained)

In [19]:
m.addLayer(result, vis_para, 'Supervised Classification')
m.addLayerControl(position = 'bottomleft')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…